# Evaluate Ensemble

This notebook combines the classification results of some models via logit-ensembling way.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.temple_eeg_script import build_dataset_for_tuab_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_confusion_matrix
from train.evaluate import calculate_confusion_matrix2
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion, draw_confusion2
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 2.0.0+cu117
cuda is available.


-----

## List up the models to check accuracy

In [4]:
model_names = [
'2pot34fr',
'36eo4dey',
'1id6fh8l',
'2c9d9csx',
'36riftdk',
'28fb1p9d',
'3u3bx062',
'ql56npap',
'1ixy3mpa',
'to3gp1w8',
]

model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:/CAUEEG/tuab_checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        print(ckpt['config']['model'])
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

2D-VGG-19
2D-VGG-19
2D-ResNeXt-50
2D-Wide-ResNet-50
1D-ResNeXt-50
1D-ResNeXt-50
1D-ResNeXt-50
2D-VGG-19
1D-ResNet-50
1D-ResNeXt-50
['2pot34fr',
 '36eo4dey',
 '1id6fh8l',
 '2c9d9csx',
 '36riftdk',
 '28fb1p9d',
 '3u3bx062',
 'ql56npap',
 '1ixy3mpa',
 'to3gp1w8']


---

## Configurations

In [5]:
task = 'exp-tuab'
eval_ensemble = True

base_repeat = 8 # 800
crop_multiple = 8
test_crop_multiple = 8

verbose = False
save_fig = True

eval_train = False
eval_val = False
eval_test = True
eval_test_multi = True

In [6]:
if eval_ensemble:
    task += '-ensem'

-----

## Evaluate each model and accumulate the logits

In [7]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    
    config['dataset_path'] = './local/dataset/tuab/'    
    config['run_mode'] = 'eval'
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    model_dict['activation'] = config.get('activation', 0)

    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure and update
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_multiple'] = crop_multiple
    config['test_crop_multiple'] = test_crop_multiple
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / crop_multiple)
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = crop_multiple
    model_dict['test_crop_multiple'] = test_crop_multiple
    
    # build dataset
    _ = build_dataset_for_tuab_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    
    # warm-up stage
    _ = check_accuracy_extended(model, test_loader, 
                                config['preprocess_test'], config, repeat=1)
    
    # train accuracy
    if eval_train:
        train_acc = check_accuracy(model, train_loader, 
                                   config['preprocess_test'], config, repeat=repeat)
        model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    if eval_val:
        val_acc = check_accuracy(model, val_loader, 
                                 config['preprocess_test'], config, repeat=repeat)
        model_dict['Validation Accuracy'] = val_acc
    
    # Test accuracy
    if eval_test:
        _ = check_accuracy_extended(model, test_loader, 
                                    config['preprocess_test'], config, repeat=repeat)
        model_dict['Test Throughput'] = _[4]
        model_dict['Test Accuracy'] = _[0]
        model_dict['Test Score'] = _[1]
        model_dict['Test Target'] = _[2]
        
        pred =  model_dict['Test Score'].argmax(axis=-1)
        target = model_dict['Test Target']
        test_confusion = calculate_confusion_matrix2(pred, target, model_dict['Test Score'].shape[-1], 
                                                     len(test_loader.dataset)) 
        test_class_wise_metrics = calculate_class_wise_metrics(test_confusion.sum(axis=0))

        for k, v in test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                model_dict[f'{k} ({c_name})'] = test_class_wise_metrics[k][c]
            
        if save_fig:
            draw_roc_curve(model_dict['Test Score'], 
                           model_dict['Test Target'], 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-ROC.pdf')

            draw_confusion(test_confusion.sum(axis=0), 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-confusion.pdf')

            draw_confusion2(test_confusion.mean(axis=0), 
                            test_confusion.std(axis=0), 
                            config['class_label_to_name'], 
                            use_wandb=False, 
                            save_path=f'local/output/imgs/{model_dict["name"]}-confusion2.pdf')

            draw_class_wise_metrics(test_confusion.sum(axis=0), 
                                    config['class_label_to_name'], 
                                    use_wandb=False, 
                                    save_path=f'local/output/imgs/{model_dict["name"]}-class-wise.pdf')
    # Multi-crop test accuracy
    if eval_test_multi:
        _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                              config['preprocess_test'], config, repeat=repeat)
        model_dict['Multi-Crop Test Throughput'] = _[4]
        model_dict['Multi-Crop Test Accuracy'] = _[0]
        model_dict['Multi-Crop Test Score'] = _[1]
        model_dict['Multi-Crop Test Target'] = _[2]
        
        pred =  model_dict['Multi-Crop Test Score'].argmax(axis=-1)
        target = model_dict['Multi-Crop Test Target']
        multi_test_confusion = calculate_confusion_matrix2(pred, target, model_dict['Test Score'].shape[-1], 
                                                           len(test_loader.dataset)) 
        multi_test_class_wise_metrics = calculate_class_wise_metrics(multi_test_confusion.sum(axis=0))

        for k, v in multi_test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                model_dict[f'Multi-Crop {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]

        if save_fig:
            draw_roc_curve(model_dict['Multi-Crop Test Score'], 
                           model_dict['Multi-Crop Test Target'], 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-roc-tta.pdf')

            draw_confusion(multi_test_confusion.sum(axis=0), 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-confusion-tta.pdf')
            
            draw_confusion2(multi_test_confusion.mean(axis=0), 
                            multi_test_confusion.std(axis=0), 
                            config['class_label_to_name'], 
                            use_wandb=False, 
                            save_path=f'local/output/imgs/{model_dict["name"]}-confusion2-tta.pdf')

            draw_class_wise_metrics(multi_test_confusion.sum(axis=0), 
                                    config['class_label_to_name'], 
                                    use_wandb=False, 
                                    save_path=f'local/output/imgs/{model_dict["name"]}-class-wise-tta.pdf') 
            
print('==== Finished ====')

- checking for 2pot34fr 2D-VGG-19 ...
- checking for 36eo4dey 2D-VGG-19 ...
- checking for 1id6fh8l 2D-ResNeXt-50 ...
- checking for 2c9d9csx 2D-Wide-ResNet-50 ...
- checking for 36riftdk 1D-ResNeXt-50 ...
- checking for 28fb1p9d 1D-ResNeXt-50 ...
- checking for 3u3bx062 1D-ResNeXt-50 ...
- checking for ql56npap 2D-VGG-19 ...
- checking for 1ixy3mpa 1D-ResNet-50 ...
- checking for to3gp1w8 1D-ResNeXt-50 ...
==== Finished ====


## Conduct ensemble

In [8]:
if eval_ensemble:
    if 'Ensemble' == model_pool[-1]['name']:
        model_pool.remove(model_pool[-1])     

    # conduct ensembling
    if eval_test:
        ensem_test_score = np.zeros_like(model_pool[0]['Test Score'])
        ensem_test_latency = 0

    if eval_test_multi:
        ensem_multi_test_score = np.zeros_like(model_pool[0]['Multi-Crop Test Score'])
        ensem_multi_test_latency = 0

    ensem_params = 0
    ensem_model_size = 0

    for model_dict in model_pool:        
        ensem_params += model_dict['num_params']
        ensem_model_size += model_dict['model size (MiB)']

        if eval_test:
            ensem_test_score += model_dict['Test Score'] / len(model_pool)
            ensem_test_latency += 1 / model_dict['Test Throughput']

        if eval_test_multi:
            ensem_multi_test_score += model_dict['Multi-Crop Test Score'] / len(model_pool)
            ensem_multi_test_latency += 1 / model_dict['Multi-Crop Test Throughput']
            
    # test accuracy
    if eval_test:
        # confusion matrix
        pred = ensem_test_score.argmax(axis=-1)
        target = model_pool[0]['Test Target']
        ensem_test_acc = 100.0 * (pred.squeeze() == target).sum() / pred.shape[0]

        ensem_test_confusion = calculate_confusion_matrix2(pred, target, ensem_test_score.shape[-1], 
                                                           len(test_loader.dataset))
        ensem_test_class_wise_metrics = calculate_class_wise_metrics(ensem_test_confusion.sum(axis=0))

        # draw
        save_path = f'local/output/imgs/{task}-ensemble-confusion.pdf' if save_fig else None
        draw_confusion(ensem_test_confusion.sum(axis=0), config['class_label_to_name'], 
                       normalize=True, use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-confusion2.pdf' if save_fig else None
        draw_confusion2(ensem_test_confusion.mean(axis=0), ensem_test_confusion.std(axis=0), 
                        config['class_label_to_name'], use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-roc.pdf' if save_fig else None
        draw_roc_curve(ensem_test_score, model_pool[0]['Test Target'], config['class_label_to_name'], 
                       use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-class-wise.pdf' if save_fig else None
        draw_class_wise_metrics(ensem_test_confusion.sum(axis=0), config['class_label_to_name'], 
                            use_wandb=False, save_path=save_path, percent=True)
        
    # multi-crop accuracy
    if eval_test_multi:
        # confusion matrix
        pred = ensem_multi_test_score.argmax(axis=-1)
        target = model_pool[0]['Multi-Crop Test Target']
        ensem_multi_test_acc = 100.0 * (pred.squeeze() == target).sum() / pred.shape[0]

        ensem_multi_test_confusion = calculate_confusion_matrix2(pred, target, ensem_multi_test_score.shape[-1], 
                                                                 len(test_loader.dataset))
        ensem_multi_test_class_wise_metrics = calculate_class_wise_metrics(ensem_multi_test_confusion.sum(axis=0))

        # draw
        save_path = f'local/output/imgs/{task}-ensemble-confusion-tta.pdf' if save_fig else None
        draw_confusion(ensem_multi_test_confusion.sum(axis=0), config['class_label_to_name'], 
                       normalize=True, use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-confusion-tta2.pdf' if save_fig else None
        draw_confusion2(ensem_multi_test_confusion.mean(axis=0), ensem_multi_test_confusion.std(axis=0), 
                        config['class_label_to_name'], use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-roc-tta.pdf' if save_fig else None
        draw_roc_curve(ensem_multi_test_score, model_pool[0]['Multi-Crop Test Target'], config['class_label_to_name'], 
                       use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-class-wise-tta.pdf' if save_fig else None
        draw_class_wise_metrics(ensem_multi_test_confusion.sum(axis=0), config['class_label_to_name'], 
                                use_wandb=False, save_path=save_path, percent=True)
        
    # summarize the ensemble results
    ensem_dict = {}

    ensem_dict['name'] = 'Ensemble'
    ensem_dict['num_params'] = ensem_params
    ensem_dict['model size (MiB)'] = ensem_model_size

    if eval_test:
        ensem_dict['Test Throughput'] = 1 / ensem_test_latency
        ensem_dict['Test Accuracy'] = ensem_test_acc

        for k, v in ensem_test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                ensem_dict[f'{k} ({c_name})'] = ensem_test_class_wise_metrics[k][c]

    if eval_test_multi:
        ensem_dict['Multi-Crop Test Throughput'] = 1 / ensem_multi_test_latency
        ensem_dict['Multi-Crop Test Accuracy'] = ensem_multi_test_acc

        for k, v in ensem_multi_test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                ensem_dict[f'Multi-Crop {k} ({c_name})'] = ensem_multi_test_class_wise_metrics[k][c]

    model_pool.append(ensem_dict)        

In [9]:
model_pool_frame = deepcopy(model_pool)

for model_dict in model_pool_frame:
    model_dict.pop('Test Score', None)
    model_dict.pop('Test Target', None)
    model_dict.pop('Multi-Crop Test Score', None)
    model_dict.pop('Multi-Crop Test Target', None)
    
pd.DataFrame(model_pool_frame).to_csv(f'local/output/{task}.csv')
pd.DataFrame(model_pool_frame)

,name,path,model,num_params,model size (MiB),seq_length,use_age,EKG,awgn,awgn_age,...,Multi-Crop Accuracy (Normal),Multi-Crop Accuracy (Abnormal),Multi-Crop Sensitivity (Normal),Multi-Crop Sensitivity (Abnormal),Multi-Crop Specificity (Normal),Multi-Crop Specificity (Abnormal),Multi-Crop Precision (Normal),Multi-Crop Precision (Abnormal),Multi-Crop F1-score (Normal),Multi-Crop F1-score (Abnormal)
0,2pot34fr,E:/CAUEEG/tuab_checkpoint\2pot34fr\checkpoint.pt,2D-VGG-19,20221250,77.218244,3000.0,conv,O,0.056878,0.178295,...,0.826087,0.826087,0.906667,0.730159,0.730159,0.906667,0.800000,0.867925,0.850000,0.793103
1,36eo4dey,E:/CAUEEG/tuab_checkpoint\36eo4dey\checkpoint.pt,2D-VGG-19,20221250,77.218244,3000.0,conv,O,0.056878,0.178295,...,0.804348,0.804348,0.880000,0.714286,0.714286,0.880000,0.785714,0.833333,0.830189,0.769231
2,1id6fh8l,E:/CAUEEG/tuab_checkpoint\1id6fh8l\checkpoint.pt,2D-ResNeXt-50,25747586,98.600127,3000.0,fc,O,0.018657,0.149320,...,0.826087,0.826087,0.933333,0.698413,0.698413,0.933333,0.786517,0.897959,0.853659,0.785714
3,2c9d9csx,E:/CAUEEG/tuab_checkpoint\2c9d9csx\checkpoint.pt,2D-Wide-ResNet-50,69079746,263.893424,5000.0,conv,O,0.030177,0.095943,...,0.844203,0.844203,0.920000,0.753968,0.753968,0.920000,0.816568,0.887850,0.865204,0.815451
4,36riftdk,E:/CAUEEG/tuab_checkpoint\36riftdk\checkpoint.pt,1D-ResNeXt-50,25659010,98.261981,3750.0,conv,X,0.080541,0.268655,...,0.822464,0.822464,0.873333,0.761905,0.761905,0.873333,0.813665,0.834783,0.842444,0.796680
5,28fb1p9d,E:/CAUEEG/tuab_checkpoint\28fb1p9d\checkpoint.pt,1D-ResNeXt-50,25659010,98.261981,3750.0,conv,X,0.080541,0.268655,...,0.847826,0.847826,0.900000,0.785714,0.785714,0.900000,0.833333,0.868421,0.865385,0.825000
6,3u3bx062,E:/CAUEEG/tuab_checkpoint\3u3bx062\checkpoint.pt,1D-ResNeXt-50,25660034,98.269455,6000.0,fc,O,0.037536,0.115381,...,0.826087,0.826087,0.920000,0.714286,0.714286,0.920000,0.793103,0.882353,0.851852,0.789474
7,ql56npap,E:/CAUEEG/tuab_checkpoint\ql56npap\checkpoint.pt,2D-VGG-19,20188482,77.090346,5000.0,conv,O,0.047940,0.031086,...,0.833333,0.833333,0.913333,0.738095,0.738095,0.913333,0.805882,0.877358,0.856250,0.801724
8,1ixy3mpa,E:/CAUEEG/tuab_checkpoint\1ixy3mpa\checkpoint.pt,1D-ResNet-50,26319234,100.726806,5000.0,fc,O,0.116489,0.096436,...,0.829710,0.829710,0.886667,0.761905,0.761905,0.886667,0.815951,0.849558,0.849840,0.803347
9,to3gp1w8,E:/CAUEEG/tuab_checkpoint\to3gp1w8\checkpoint.pt,1D-ResNeXt-50,25659010,98.261981,3750.0,conv,X,0.080541,0.268655,...,0.833333,0.833333,0.866667,0.793651,0.793651,0.866667,0.833333,0.833333,0.849673,0.813008


##### 

##### 